In [ ]:
import pandas as pd
act = pd.read_csv("D:/000-Master/Metropia_UserClustering/sample_activity_Oct-22-2018.csv")
act.rename(columns = {'Unnamed: 0':'index'},inplace=True)
loc = pd.read_csv("D:/000-Master/Metropia_UserClustering/sample_location_Oct-22-2018.csv")
loc.rename(columns = {'Unnamed: 0':'index'},inplace=True)
city_list = (dict(loc['city'].value_counts()))

act['start_date'] = act['local_start_time'].astype(str).str[:10]
act['local_start_time'] = act['local_start_time'].astype(str).str[:19]
act['local_start_time'] = pd.to_datetime(act['local_start_time'])
act['local_start_hour'] = act['local_start_time'].dt.hour

act['local_end_time'] = act['local_end_time'].astype(str).str[:19]
act['local_end_time'] = pd.to_datetime(act['local_end_time'])

import numpy as np
cols = ['loc_id','center_lat','center_lon']
df = loc[cols]

df.rename(columns = {'loc_id':'orig_loc_id'},inplace=True)
x = pd.merge(act, df, on="orig_loc_id")
x.rename(columns = {'center_lat':'orig_lat',
                   'center_lon':'orig_lon'},inplace=True)

df.rename(columns = {'orig_loc_id':'dest_loc_id'},inplace=True)
x = pd.merge(x, df, on="dest_loc_id")
x.rename(columns = {'center_lat':'dest_lat',
                   'center_lon':'dest_lon'},inplace=True)
act = x

from datetime import datetime, date, time
def weekday(data):
    dt = datetime.strptime(data, "%Y-%m-%d")
    tt = dt.timetuple()
    return tt[6]
    
act['day'] = (act['start_date']).apply(weekday) # weekday (0 = Monday)

act.head()

In [ ]:
### 1. Distance Matrix ####
from sklearn.cluster import KMeans
km = KMeans(120, init='k-means++') # initialize
X = data[['center_x','center_y']]
km.fit(X)
label = km.predict(X) # classify into 120 clusters
data['label'] = label

max_dist = [] # maximum within-group distance of group j
avg_dist = [] # average distance
j = 0
for i in set(data['label']):    
    one = data.loc[data['label'] == i]
    one_1 = one.loc[one.index.repeat(len(one))].reset_index(drop=True) # reset index
    one_2 = one
    one_2 = pd.concat([one_2]*len(one_2), ignore_index=True)
    one_2.rename(columns = {'loc_id':'loc_id_2','center_y': 'center_y_2','center_x':'center_x_2'},inplace=True)
    one_3 = one_1.join(one_2, lsuffix='_one_1', rsuffix='_one_2') # IMPORTANT!!!
    from sklearn.metrics.pairwise import pairwise_distances 
    from sklearn.cluster import DBSCAN 
    from geopy.distance import vincenty
    # unit in km
    one_3['distance'] = np.sqrt(np.square(one_3['center_y']-one_3['center_y_2'])+np.square(one_3['center_x']-one_3['center_x_2']))
    one_distance_matrix = one_3.pivot_table(index=['loc_id'],columns=['loc_id_2'],values='distance')
    max_dist.append(one_distance_matrix.values.max())
    avg_dist.append(one_distance_matrix.values.mean())
    j = j + 1
    
max_dist = pd.DataFrame(max_dist) # max distance winthin group
max_dist.columns = ['distance']
max_dist.head()
avg_dist = pd.DataFrame(avg_dist) # average distance within group
avg_dist.columns = ['distance']
avg_dist.head()

plt.hist(avg_dist['distance'], bins=25, alpha=0.6, color='g') #hist 
title = "Distribution of Average Distance Within Group"
plt.xlabel("Distance/km")
plt.ylabel("Number of Groups")
plt.title(title)
plt.show()

In [1]:
### Three Algorithms ####


In [ ]:
### 2. Number of Users in a Group ####


In [ ]:
### 2. Number of Users in a Group ####

In [ ]:
### 3. # of groups a user have ####